# Mapa con choropleth_mapbox y Dash

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point, Polygon
import geopandas as gpd

In [6]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 

from shapely.geometry import Point, Polygon 
import geopandas as gpd
from shapely.geometry import shape
import descartes
import json

In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html

from datetime import datetime as dt
import plotly.graph_objects as go

In [8]:
map_nepal = gpd.read_file('.\Data\downloadeddata.geojson')
map_nepal

,id,F_ID,N_ID,NAME,LEVEL,DISTRICT,districtId,provinceId,geometry
0,None,104,bhaktapur-bhaktapur,Bhaktapur,Nagarpalika,Bhaktapur,BHAKTAPUR,3,"POLYGON ((85.44459 27.66441, 85.44378 27.66359..."
1,None,105,madhyapur -thimi-bhaktap,Madhyapur Thimi,Nagarpalika,Bhaktapur,BHAKTAPUR,3,"POLYGON ((85.36314 27.66892, 85.35419 27.66769..."
2,None,343,kathmandu-kathmandu,Kathmandu,Mahanagarpalika,Kathmandu,KATHMANDU,3,"POLYGON ((85.28577 27.68983, 85.28414 27.69270..."
3,None,344,kirtipur-kathmandu,Kirtipur,Nagarpalika,Kathmandu,KATHMANDU,3,"POLYGON ((85.25319 27.64555, 85.25238 27.65416..."
4,None,396,chitwan -national -park-,Chitwan National Park,national-park,Makwanpur,MAKWANPUR,3,"POLYGON ((84.71727 27.56355, 84.71727 27.55945..."
...,...,...,...,...,...,...,...,...,...
761,None,550,gadhimai-rautahat,Gadhimai,Gaunpalika,Rautahat,RAUTAHAT,2,"POLYGON ((85.33220 26.90879, 85.33138 26.91084..."
762,None,549,madhav -narayan-rautahat,Madhav Narayan,Gaunpalika,Rautahat,RAUTAHAT,2,"POLYGON ((85.29554 26.86779, 85.29147 26.87271..."
763,None,547,dawahi -gonahi-rautahat,Dawahi Gonahi,Gaunpalika,Rautahat,RAUTAHAT,2,"POLYGON ((85.24342 26.91986, 85.24342 26.92191..."
764,None,545,maulapur-rautahat,Maulapur,Gaunpalika,Rautahat,RAUTAHAT,2,"POLYGON ((85.19129 26.95102, 85.19374 26.95553..."


In [9]:
with open('.\Data\downloadeddata.geojson') as f:
    map_nepal_json = json.load(f)
# features = map_nepal_json['features'][0]
# features
#map_nepal_json

In [10]:
# # Limpiamos
# map_nepal.drop(["id", "F_ID", "N_ID", "LEVEL", "districtId", "provinceId"], axis=1, inplace=True)

In [11]:
# Creamos 1 columna unica con la suma de Name y distrito, porque hay municipios con el mismo nombre, pero que están en
# dos distritos diferentes

In [12]:
map_nepal["suma"]=map_nepal["NAME"] + map_nepal["DISTRICT"]

In [13]:
# Cruce de Municipios y Distritos de Geojason con Open Data.
# Incorpora el Vdcmun_id y la Geometry
# El nombre que prevalece es el de Geojason 
# Prueba2 solo tiene los 111 registros de Open Data. Cuando lo juntemos luego, llegarán a los 766
df = pd.read_csv('./Data/prueba2.csv', sep=';')
df

,NAME_Geojason,DISTRITO_Geojason,vdcmun_id,geometry_Geojason
0,Benighat Rorang,Dhading,3001,"POLYGON ((84.69528102102205 27.72139708172487,..."
1,Dhunibesi,Dhading,3002,"POLYGON ((85.22224194389047 27.72672697300562,..."
2,Gajuri,Dhading,3003,"POLYGON ((84.90785722020081 27.69720757514299,..."
3,Galchi,Dhading,3004,"POLYGON ((84.90785722020081 27.8464445310041, ..."
4,Gangajamuna,Dhading,3005,"POLYGON ((84.82478146419992 28.06415009024105,..."
...,...,...,...,...
106,Lisangkhu Pakhar,Sindhupalchowk,2309,"POLYGON ((85.81924713162238 27.6644082441845, ..."
107,Melamchi,Sindhupalchowk,2310,"POLYGON ((85.6522811514245 27.84767450591504, ..."
108,Panchpokhari Thangpal,Sindhupalchowk,2311,"POLYGON ((85.68485987926799 28.15639820856179,..."
109,Sunkoshi,Sindhupalchowk,2312,"POLYGON ((85.87707437354457 27.74640657158071,..."


In [14]:
# Eliminamos la geometria, porque es de tipo object y no nos vale para nada:
df.drop(["geometry_Geojason"], axis=1, inplace=True)

In [15]:
# Creamos la misma columna "suma", que contatena municipio y distrito

df["suma"]=df["NAME_Geojason"] +df["DISTRITO_Geojason"]
df

,NAME_Geojason,DISTRITO_Geojason,vdcmun_id,suma
0,Benighat Rorang,Dhading,3001,Benighat RorangDhading
1,Dhunibesi,Dhading,3002,DhunibesiDhading
2,Gajuri,Dhading,3003,GajuriDhading
3,Galchi,Dhading,3004,GalchiDhading
4,Gangajamuna,Dhading,3005,GangajamunaDhading
...,...,...,...,...
106,Lisangkhu Pakhar,Sindhupalchowk,2309,Lisangkhu PakharSindhupalchowk
107,Melamchi,Sindhupalchowk,2310,MelamchiSindhupalchowk
108,Panchpokhari Thangpal,Sindhupalchowk,2311,Panchpokhari ThangpalSindhupalchowk
109,Sunkoshi,Sindhupalchowk,2312,SunkoshiSindhupalchowk


In [16]:
df['district_id']=df['vdcmun_id'].astype(str).str.extract(r"(\d{2})")
df['district_id']=np.int64(df['district_id'])

In [17]:
# Ahora juntamos los dos dataframes por la columna comun que es "suma" y conseguimos los 766
total_map = pd.merge(map_nepal,df, how='outer',on='suma')
# total_map.head()

In [18]:
# total_map2 = pd.merge(map_nepal_json,df, how='outer',on='suma')
# total_map2

In [19]:
# Limpiamos las dos columnas que nos nos valen ya para nada
total_map.drop(["NAME_Geojason", "DISTRITO_Geojason"], axis=1, inplace=True)

In [20]:
# total_map.isnull().sum()

In [21]:
total_map.head()

,id,F_ID,N_ID,NAME,LEVEL,DISTRICT,districtId,provinceId,geometry,suma,vdcmun_id,district_id
0,None,104,bhaktapur-bhaktapur,Bhaktapur,Nagarpalika,Bhaktapur,BHAKTAPUR,3,"POLYGON ((85.44459 27.66441, 85.44378 27.66359...",BhaktapurBhaktapur,NaN,NaN
1,None,105,madhyapur -thimi-bhaktap,Madhyapur Thimi,Nagarpalika,Bhaktapur,BHAKTAPUR,3,"POLYGON ((85.36314 27.66892, 85.35419 27.66769...",Madhyapur ThimiBhaktapur,NaN,NaN
2,None,343,kathmandu-kathmandu,Kathmandu,Mahanagarpalika,Kathmandu,KATHMANDU,3,"POLYGON ((85.28577 27.68983, 85.28414 27.69270...",KathmanduKathmandu,NaN,NaN
3,None,344,kirtipur-kathmandu,Kirtipur,Nagarpalika,Kathmandu,KATHMANDU,3,"POLYGON ((85.25319 27.64555, 85.25238 27.65416...",KirtipurKathmandu,NaN,NaN
4,None,396,chitwan -national -park-,Chitwan National Park,national-park,Makwanpur,MAKWANPUR,3,"POLYGON ((84.71727 27.56355, 84.71727 27.55945...",Chitwan National ParkMakwanpur,NaN,NaN


In [23]:
total_map.shape

(766, 12)

In [26]:
total_map[total_map['district_id']==20]

,id,F_ID,N_ID,NAME,LEVEL,DISTRICT,districtId,provinceId,geometry,suma,vdcmun_id,district_id
291,None,648,hariharpurgadhi-sindhuli,Hariharpurgadhi,Gaunpalika,Sindhuli,SINDHULI,3,"POLYGON ((85.49183 27.15438, 85.49427 27.16053...",HariharpurgadhiSindhuli,2004.0,20.0
292,None,649,marin-sindhuli,Marin,Gaunpalika,Sindhuli,SINDHULI,3,"POLYGON ((85.64577 27.14290, 85.64332 27.14659...",MarinSindhuli,2006.0,20.0
293,None,650,ghyanglekh-sindhuli,Ghyanglekh,Gaunpalika,Sindhuli,SINDHULI,3,"POLYGON ((85.80296 27.29296, 85.78178 27.29788...",GhyanglekhSindhuli,2002.0,20.0
294,None,651,kamalamai-sindhuli,Kamalamai,Nagarpalika,Sindhuli,SINDHULI,3,"POLYGON ((86.05300 27.01867, 86.05137 27.01867...",KamalamaiSindhuli,2005.0,20.0
295,None,652,sunkoshi-sindhuli,Sunkoshi,Gaunpalika,Sindhuli,SINDHULI,3,"POLYGON ((85.77608 27.39791, 85.77852 27.41021...",SunkoshiSindhuli,2008.0,20.0
296,None,653,golanjor-sindhuli,Golanjor,Gaunpalika,Sindhuli,SINDHULI,3,"POLYGON ((86.18331 27.25196, 86.18413 27.24909...",GolanjorSindhuli,2003.0,20.0
297,None,654,dudhouli-sindhuli,Dudhouli,Nagarpalika,Sindhuli,SINDHULI,3,"POLYGON ((86.26232 26.91863, 86.25987 26.92109...",DudhouliSindhuli,2001.0,20.0
298,None,655,tinpatan-sindhuli,Tinpatan,Gaunpalika,Sindhuli,SINDHULI,3,"POLYGON ((86.19635 27.21670, 86.19716 27.21096...",TinpatanSindhuli,2009.0,20.0
299,None,656,phikkal-sindhuli,Phikkal,Gaunpalika,Sindhuli,SINDHULI,3,"POLYGON ((86.25092 27.12937, 86.24929 27.13183...",PhikkalSindhuli,2007.0,20.0
